In [1]:
import pandas as pd
import random

# 예시 데이터 생성
data = {
    'line_id': ['KFBK'] * 5,
    'root_lot_id': [f'A3HTN{i}' for i in range(1, 6)],
    'lot_id': [f'A3HTN{i}.1' for i in range(1, 6)],
    'wafer_id': [str(random.randint(1, 25)) for _ in range(5)],
    'dcop_id': ['N03SP1HA'] * 5,
    'item_id': ['P019_ISTBY_96KB_075IWL_MOL'] * 5,
    'hold_infor': ['PCT=5 (1)'] * 5,
    'hold_time': [
        '2024-04-24-1:23',
        '2024-04-24-2:45',
        '2024-04-25-3:10',
        '2024-04-25-4:55',
        '2024-04-26-5:30'
    ],
    'section': ['FINAL'] * 5,
    'prod_mast': ['S5E5535A'] * 5,
    'pro_id': ['LN03GAPM'] * 5,
    'pro_plan_id': ['KNNY'] * 5,
    'step_id': ['NU995000'] * 5,
    'folder_nan': [None] * 5,
    'folder_dat': [None] * 5
}

# 데이터프레임 생성
merged_df = pd.DataFrame(data)

# 데이터프레임 출력
display(merged_df)

,line_id,root_lot_id,lot_id,wafer_id,dcop_id,item_id,hold_infor,hold_time,section,prod_mast,pro_id,pro_plan_id,step_id,folder_nan,folder_dat
0,KFBK,A3HTN1,A3HTN1.1,13,N03SP1HA,P019_ISTBY_96KB_075IWL,PCT=5 (1),2024-04-24-1:23,FINAL,S5E5535A,LN03GAPM,KNNY,NU995000,None,None
1,KFBK,A3HTN2,A3HTN2.1,20,N03SP1HA,P019_ISTBY_96KB_075IWL,PCT=5 (1),2024-04-24-2:45,FINAL,S5E5535A,LN03GAPM,KNNY,NU995000,None,None
2,KFBK,A3HTN3,A3HTN3.1,8,N03SP1HA,P019_ISTBY_96KB_075IWL,PCT=5 (1),2024-04-25-3:10,FINAL,S5E5535A,LN03GAPM,KNNY,NU995000,None,None
3,KFBK,A3HTN4,A3HTN4.1,20,N03SP1HA,P019_ISTBY_96KB_075IWL,PCT=5 (1),2024-04-25-4:55,FINAL,S5E5535A,LN03GAPM,KNNY,NU995000,None,None
4,KFBK,A3HTN5,A3HTN5.1,8,N03SP1HA,P019_ISTBY_96KB_075IWL,PCT=5 (1),2024-04-26-5:30,FINAL,S5E5535A,LN03GAPM,KNNY,NU995000,None,None


In [12]:
import os
import pandas as pd

# 날짜만 추출하는 함수 (YYYYMMDD 형태)
def extract_date(hold_time):
    # 예: '2024-04-25-3:10' → '20240425'
    return ''.join(hold_time.split('-')[:3])

# main.csv의 priority 조건 함수
def get_priority(hold_infor):
    hold_infor = str(hold_infor)
    oos = any(x in hold_infor for x in ['PCT', 'MCT'])
    ooc = any(x in hold_infor for x in ['USL', 'LSL'])
    if oos:
        return 'OOS'
    elif ooc:
        return 'OOC'
    else:
        return ''

# sub.csv의 module 조건 함수
def get_modules(item_id):
    item_id = str(item_id)
    modules = []
    if any(x in item_id for x in ['BEOL', 'BEOL1', 'BEOL2']):
        modules.append('BEOL')
    if any(x in item_id for x in ['MOL', 'MOL1', 'MOL2']):
        modules.append('MOL')
    if any(x in item_id for x in ['RPMG', 'RPMG1', 'RPMG2']):
        modules.append('RPMG')
    return modules

# hold_time에서 날짜만 추출해서 새로운 컬럼 생성
merged_df['date_folder'] = merged_df['hold_time'].apply(lambda x: extract_date(str(x)))

# 날짜별로 반복
for date in merged_df['date_folder'].unique():
    date_folder_path = f'./{date}'
    os.makedirs(date_folder_path, exist_ok=True)
    
    # 해당 날짜의 데이터프레임 추출
    date_df = merged_df[merged_df['date_folder'] == date]
    
    # lot_id별로 반복
    for idx, lot_id in enumerate(date_df['lot_id'].unique(), 1):
        folder_name = f'ETAOCAP{date}_{idx}'
        lot_folder_path = os.path.join(date_folder_path, folder_name)
        os.makedirs(lot_folder_path, exist_ok=True)
        
        # 해당 lot_id에 맞는 데이터 추출
        filtered_df = date_df[date_df['lot_id'] == lot_id]
        row = filtered_df.iloc[0]  # 대표 row 하나 사용

        # main.csv 데이터 생성
        main_data = {
            'title': f'{lot_id} OCAP 확인 도움 부탁드립니다.',
            'priority': get_priority(row['hold_infor']),
            'prod stage': '양산',
            'issue category': 'DC OCAP',
            'line id': row['line_id'],
            'process': row['pro_id'],
            'process id': row['pro_plan_id'],
            'part id': row['prod_mast'],
            'person in charge': 'oakgun.park, taegun.park',
            'fst issue empid': 'yw.kang'
        }
        main_df = pd.DataFrame([main_data])
        main_df.to_csv(os.path.join(lot_folder_path, 'main.csv'), index=False)

        # sub.csv 데이터 생성
        sub_rows = []
        modules = get_modules(row['item_id'])
        # 여러 module이 해당되면 각각 한 줄씩 생성
        for seq, module in enumerate(modules, 1):
            sub_row = {
                'sub issue id': f'{folder_name}_{seq}',  # ETAOCAP폴더명_수열
                'status': '발의',
                'answer dept': '(YE) MODULE',
                'module': module,
                'dc step': row['step_id'],
                'dcop': row['dcop_id'],
                'dc item': row['item_id'],
                'reFormat': 'N',
                'layer': '',
                'fab step': '',
                'fab item': '',
                'dc dirctn': 'OOS',
                'fab dirctn': '',
                'slope': '',
                'person in charge': 'sungju.moon, jiho.song',
                'deadline': '',
                'fst issue empid': 'yw.kang'
            }
            sub_rows.append(sub_row)
        # 만약 해당 module이 없으면 한 줄만 생성 (module 공백)
        if not modules:
            sub_row = {
                'sub issue id': f'{folder_name}_1',
                'status': '발의',
                'answer dept': '(YE) MODULE',
                'module': '',
                'dc step': row['step_id'],
                'dcop': row['dcop_id'],
                'dc item': row['item_id'],
                'reFormat': 'N',
                'layer': '',
                'fab step': '',
                'fab item': '',
                'dc dirctn': 'OOS',
                'fab dirctn': '',
                'slope': '',
                'person in charge': 'sungju.moon, jiho.song',
                'deadline': '',
                'fst issue empid': 'yw.kang'
            }
            sub_rows.append(sub_row)
        sub_df = pd.DataFrame(sub_rows)
        sub_df.to_csv(os.path.join(lot_folder_path, 'sub.csv'), index=False)

        # lotwf.csv 데이터 생성
        # 여러 행이 있을 수 있으니, filtered_df 전체를 사용
        lotwf_rows = []
        for _, wf_row in filtered_df.iterrows():
            lotwf_row = {
                'process id': wf_row['pro_plan_id'],         # merged_df의 pro_plan_id
                'root_lot_id': wf_row['root_lot_id'],        # merged_df의 root_lot_id
                'lot_id': wf_row['lot_id'],                  # merged_df의 lot_id
                'wafer_id': wf_row['wafer_id'],              # merged_df의 wafer_id
                'expect_yld_effect_value': '',               # 공백
                'category': '',                              # 공백
                'cause_step_seq': '',                        # 공백
                'cause_factor_type': '',                     # 공백
                'cause_factor': wf_row['hold_infor'],        # merged_df의 hold_infor
                'true_loss/gain': '',                        # 공백
                'expect_fabout_week': '',                    # 공백
                'remain wfqty': ''                           # 공백
            }
            lotwf_rows.append(lotwf_row)
        lotwf_df = pd.DataFrame(lotwf_rows)
        lotwf_df.to_csv(os.path.join(lot_folder_path, 'lotwf.csv'), index=False)

                # content.html 생성 (lot_id, wafer_id 테이블 포함)
        html_path = os.path.join(lot_folder_path, 'content.html')
        wafer_table_rows = ""
        for _, wf_row in filtered_df.iterrows():
            wafer_table_rows += f"<tr><td>{wf_row['lot_id']}</td><td>{wf_row['wafer_id']}</td></tr>\n"

        html_content = f"""
        <html>
        <head>
            <meta charset="utf-8">
            <style>
                body {{
                    font-family: '맑은 고딕', Arial, sans-serif;
                    margin: 30px;
                }}
                h2 {{
                    color: #2c3e50;
                }}
                table {{
                    border-collapse: collapse;
                    width: 50%;
                    margin-bottom: 30px;
                }}
                th, td {{
                    border: 1px solid #888;
                    padding: 8px 12px;
                    text-align: center;
                }}
                th {{
                    background-color: #f2f2f2;
                }}
                .info {{
                    background-color: #f9f9f9;
                    border-left: 4px solid #3498db;
                    padding: 10px 20px;
                    margin-bottom: 20px;
                }}
            </style>
        </head>
        <body>
            <h2>OCAP LOT 알림</h2>
            <div class="info">
                아래의 랏(lot)들은 <b>OCAP</b>에 걸린 랏입니다.<br>
                <b>sub.csv</b>의 <b>module</b> 담당자 분들께서는 반드시 확인해주시고, 상단 세부사항에 응답 부탁드립니다.
            </div>
            <table>
                <tr>
                    <th>Lot ID</th>
                    <th>Wafer ID</th>
                </tr>
                {wafer_table_rows}
            </table>
            <div>
                문의사항이 있으시면 담당자에게 연락해주시기 바랍니다.<br>
                감사합니다.
            </div>
        </body>
        </html>
        """

        with open(html_path, 'w', encoding='utf-8') as f:
            f.write(html_content)